# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [377]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
import pylab as plt

import warnings
warnings.filterwarnings('ignore')

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0
    #random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3
    #random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
7000


## Пишем свой класс для SVM

In [415]:
import numpy as np
from random import randint
import random


#np.random.seed(42)
#random.seed(42)


class MySVM(object):
    def __init__(self, C=10000, eps=0.000001):
        self.C = C # regularization constant
        self.eps = eps

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
        #return 1/(1 + np.exp(-self.f(x))) #sigmoid
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        return 1.0 * sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.a(x)])
        #return - answer * np.log(self.a(x)) #cross-entropy

    # hinge loss derivative
    def der_loss(self, x, answer):
        return -answer if answer * self.a(x) < 1 else 0
        #return -answer * (1 - self.a(x)) #cross-entropy

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        loss = []
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 * 0.9 ** np.log(k + 2)

            # w update
            self.w = self.w - step * (self.der_loss(x, y) * x) - self.der_reg()
            
            # w_0 update
            self.w0 = self.w0 - step * self.der_loss(x, y)
            
            if k % 10 == 0:
                loss.append(self.loss(x, y))
            
        return loss

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [425]:
model = MySVM(eps = 0.1)
loss = model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.21383145 -1.60362333 -2.18849601  1.43393811  2.77295861 -0.69527784
 -1.6929165   0.04610358  1.42328733 -2.27024126  2.43633626 -1.0952698
 -2.08419695  0.14181441  5.60949035 -2.51500621 -2.08215468 -0.98956354
 -3.70693368  4.25954653] 6.47479011416516


In [426]:
predictions = model.predict(X_test)

In [427]:
print(predictions)

[0 0 1 ... 1 0 1]


In [428]:
print(y_test, len(y_test), sum(y_test))

[0 0 0 ... 0 0 0] 3000 1516


In [429]:
print(len(predictions), sum(predictions))

3000 2594


In [430]:
print(accuracy_score(y_test, predictions.round()))

0.5686666666666667


In [431]:
accuracy = []
C = 1
while C <= 10000:
    model = MySVM(C)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, predictions.round()))
    C *= 10

In [432]:
np.round(accuracy, decimals=4)

array([0.4947, 0.55  , 0.6133, 0.616 , 0.5973])

## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [386]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(C = 10000, verbose=True)

In [387]:
SVC.fit(X_train, y_train)

[LibLinear]

LinearSVC(C=10000, verbose=True)

In [388]:
SVC.coef_

array([[-0.13228842, -0.00152288, -0.06937925,  0.02130071,  0.08460007,
        -0.02521742,  0.07655573, -0.14435871,  0.01778339, -0.12180991,
        -0.00306177,  0.09249947, -0.00452312, -0.14713378,  0.18850398,
        -0.06930656, -0.14373887,  0.20295597,  0.05167322,  0.09250697]])

In [389]:
pred = SVC.predict(X_test)

In [390]:
accuracy_score(y_test, pred)

0.7506666666666667

In [391]:
C = 1
SVC_accuracy = []
while C <= 10000:
    SVC = LinearSVC(C=C, max_iter = 10000)
    SVC.fit(X_train, y_train)
    pred = SVC.predict(X_test)
    SVC_accuracy.append(accuracy_score(y_test, pred))
    C *= 10

In [392]:
np.round(SVC_accuracy, decimals=4)

array([0.821, 0.821, 0.758, 0.772, 0.775])